In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os

In [ ]:
## specify areas of interest / districts and metadata
## URL method accessed an UrbanShift city's boundaries and uses information from file name and geoBoundaries properties ("shapeName") to create properties for output file
#URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM1/boundary-CHN-Ningbo-ADM1.geojson'
URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM2/boundary-CRI-San_Jose-ADM2.geojson'
DistrictsGJ = requests.get(URL).json()
Districts = geemap.geojson_to_ee(DistrictsGJ)
#Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
DistrictsProjCRS = Districts.geometry().projection().crs()

print(DistrictsProjCRS.getInfo())
print(Districts.limit(1).getInfo())

In [ ]:
# extract area properties from standarized filename
# https://note.nkmk.me/en/python-split-rsplit-splitlines-re/ 
basename = os.path.splitext(os.path.basename(URL))[0]
AOIname = basename.split('-',1)[1].rsplit('-',1)[0]
#ADMlevel = basename.split('-')[-1]

Areaofinterest = AOIname ## 3-letter country abreviation - city name with underscore for spaces, e.g. "ETH-Addis_Ababa"
#unitofanalysis = ADMlevel ## options: "ADM0" (country), "ADM1" (state), "Metro" (metropolitan region), "ADM2" (municipality), "ADM3" (subcity/locality), "ADM4"(ward/neighborhood), ideally align with https://www.geoboundaries.org/index.html#getdata

print(Areaofinterest)
#print(unitofanalysis)

In [ ]:
## create map
Map = geemap.Map(height="400px")
Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=12)

In [ ]:
## GLAD land cover change datasets
LCLUC2000 = ee.Image('projects/glad/GLCLU2020/LCLUC_2000')
LCLUC2020 = ee.Image('projects/glad/GLCLU2020/LCLUC_2020')
LCLUCchange = ee.Image('projects/glad/GLCLU2020/LCLUC')

# simplify the taxonomy 
def simplifyLC(im):
    im = ee.Image(im)
    im=(im.where(im.eq(0),0)
    .where(im.gte(1).And(im.lte(24)),1)
    .where(im.gte(25).And(im.lte(41)),2)
    .where(im.gte(42).And(im.lte(48)),3)
    .where(im.gte(100).And(im.lte(124)),4)
    .where(im.gte(125).And(im.lte(148)),5)
    .where(im.gte(200).And(im.lte(207)),6)
    .where(im.eq(241),7)
    .where(im.eq(244),8)
    .where(im.eq(250),9)
    .where(im.eq(255),10)
           )
    return im.updateMask(im.lte(9).And(im.gte(0)))

LC2000 = simplifyLC(LCLUC2000)
LC2020 = simplifyLC(LCLUC2020)

LCproj = LC2020.projection().nominalScale()
print(LC2000.getInfo())
print(LCproj.getInfo())

In [ ]:
## define styling and legend 

LCLUCclasses = [
"Bare Ground",
"Short Vegetation",
"Forest",
"Tall Forest (20m+)",
"Wetland - Short Vegetation",
"Wetland - Forest",
"Water",
"Snow/Ice",
"Cropland",
"Built-up Area"
  ]

LCLUCcolors = [
"FEFECC",
"B9B91E",
"347834",
"0D570D",
"88CAAD",
"589558",
"6BAED6",
"ACD1E8",
"FFF183",
"E8765D"
  ]

# add to map
vizLC = {'min':0, 'max':9, 'palette': LCLUCcolors}
Map.addLayer(LC2000,vizLC,"LCLUC 2000 (UMD)")
Map.addLayer(LC2020,vizLC,"LCLUC 2020 (UMD)")

Map.add_legend(legend_keys=LCLUCclasses, legend_colors=LCLUCcolors, position='bottomleft')


In [ ]:
# create images with habitat and non-habitat classes in each year

def habitatLC(im):
    im = ee.Image(im)
    im = (im.where(im.eq(0),0)
            .where(im.gte(7).And(im.lte(9)),0)
            .where(im.gte(1).And(im.lte(6)),1)
           )
    return im.updateMask(im.gte(0).And(im.lte(1)))

Hab2000 = habitatLC(LC2000)
Hab2020 = habitatLC(LC2020)

print(Hab2000.getInfo())

vizHab = {'min':0, 'max':1, 'palette': ['black','white']}

Map.addLayer(Hab2000,vizHab,"Habitat 2000")
Map.addLayer(Hab2020,vizHab,"Habitat 2020")

In [ ]:
# create image with habitat gains and losses between start and end years
def change(start, end):
    im = (start.where(start.eq(0).And(end.eq(0)),0)
            .where(start.eq(1).And(end.eq(1)),0)
            .where(start.eq(0).And(end.eq(1)),1)
            .where(start.eq(1).And(end.eq(0)),-1)
           )
    return im.updateMask(im.gte(-1).And(im.lte(1)).And(im.neq(0)))

HabChange = change(Hab2000,Hab2020)
print(HabChange.getInfo())

vizCh = {'min':-1, 'max':1, 'palette': ['red','white','green']}
Map.addLayer(HabChange,vizCh,'Habitat change')

In [ ]:
## Define masks for count operations
NonHab2000masked = Hab2000.updateMask(Hab2000.eq(0))
HabGain2020masked = HabChange.updateMask(HabChange.eq(1))
Habtypes2020 = LC2020.updateMask(Hab2020.eq(1))
NewHabtypes2020 = LC2020.updateMask(HabChange.eq(1))
#Map.addLayer(NewHabtypes2020,vizLC,"mask")

In [ ]:
## reduce image counts to featurecollection of districts

histo=HabGain2020masked.reduceRegions(
  reducer= ee.Reducer.count().setOutputs(["count_gain2020"]),
  collection= Districts, 
  scale= 30, 
  tileScale= 1
)

histo=NonHab2000masked.reduceRegions(
  reducer= ee.Reducer.count().setOutputs(["count_nonhab2000"]), 
  collection= histo, 
  scale= 30, 
  tileScale= 1
)

histo=Habtypes2020.reduceRegions(
  reducer= ee.Reducer.countDistinctNonNull().setOutputs(["countD_habtypes2020"]), 
  collection= histo, 
  scale= 30, 
  tileScale= 1
)

histo=NewHabtypes2020.reduceRegions(
  reducer= ee.Reducer.countDistinctNonNull().setOutputs(["countD_newhabtypes2020"]), 
  collection= histo, 
  scale= 30, 
  tileScale= 1
)

print('histo:', histo.limit(1).getInfo())

In [ ]:
# convert counts to indicators and create new featurecollection

def count_to_stats(feat):
    feat=ee.Feature(feat)
    
    HabRestPct = feat.getNumber('count_gain2020').divide(feat.getNumber('count_nonhab2000'))
    HabTypesRestPct = feat.getNumber('countD_newhabtypes2020').divide(feat.getNumber('countD_habtypes2020'))
    #HabRestArea = feat.getNumber('count_gain2020').multiply(ee.Number(30)).multiply(ee.Number(30)).multiply(ee.Number(0.000001))
    HabRestArea = feat.getNumber('count_gain2020').multiply(ee.Number(LCproj)).multiply(ee.Number(LCproj)).multiply(ee.Number(0.000001))

    FeatArea = feat.area(0.001).multiply(0.000001)
    cityID = Areaofinterest
    geo_level = feat.getString("shapeID").split('-').getString(1)
    #geo_level = unitofanalysis
    #geo_name = feat.getString("Sub_City").cat(ee.String("-")).cat(feat.getString("Woreda"))
    #geo_name = feat.getString("city_name_viz").split(' ').join('_')
    geo_name = feat.getString("shapeName").split(' ').join('_')
    geo_id = ee.String(cityID+"-").cat(geo_name)
    year = 2020
    source = "GLAD LCLUC"
    
    return feat.set({
        'SICB-7A-HabAreaRestoredPct2000-2020': HabRestPct, 
        'TotalareaKM2': FeatArea,
        'SICB-7B-HabTypesRestoredPct2000-2020': HabTypesRestPct,
        'HabRestAreaKM2-2000-2020': HabRestArea,
        'geo_level': geo_level,
        'geo_name': geo_name,
        'geo_id': geo_id,
        'year':year,
        'source':source,
    })

HabGainPcts=histo.map(count_to_stats)

print(HabGainPcts.limit(1).getInfo())

In [ ]:
## render on map percent tree cover by class from feature collection

Tpctfills = ee.Image().paint(**{'featureCollection': HabGainPcts,'color': 'SICB-7B-HabTypesRestoredPct2000-2020'})
Tpctfills2 = ee.Image().paint(**{'featureCollection': HabGainPcts,'color': 'SICB-7A-HabAreaRestoredPct2000-2020'})

fillspalette = ['red', 'green']
Map.addLayer(Districts,{},"districts")
Map.addLayer(Tpctfills, {'palette': fillspalette,'min':0,'max':1}, '% habitat types restored (2000-2020)', True, 0.65)
Map.addLayer(Tpctfills2, {'palette': fillspalette,'min':0,'max':0.10}, '% habitat restored (2000-2020)', True, 0.65)
Map

In [ ]:
# keep only properties desired for the output file. 
HabGainPcts=HabGainPcts.select([
    'SICB-7A-HabAreaRestoredPct2000-2020', 
    'TotalareaKM2',
    'SICB-7B-HabTypesRestoredPct2000-2020', 
    'HabRestAreaKM2-2000-2020',
    'geo_level',
    'geo_name',
    'geo_id',
    'year',
    'source'])

df = geemap.ee_to_geopandas(HabGainPcts)
df = df.sort_values(by=['SICB-7B-HabTypesRestoredPct2000-2020'],axis='index',ascending=False)
df

In [ ]:
import matplotlib.pyplot as plt
plt.close("all")
import pandas as pd


df.plot.bar(x="geo_name", y="SICB-7B-HabTypesRestoredPct2000-2020",figsize=(16, 6))


Download/export options 
We should probably adjust these to instead publish output files to our AWS bucket. 

In [ ]:
# Download attribute table as a CSV
url = HabGainPcts.getDownloadURL(
    filetype="csv",
    filename="HabGainPcts",
)
print("CSV",url)

# Download attribute table as a GeoJSON
url = HabGainPcts.getDownloadURL(
    filetype="GEOJSON",
    filename="HabGainPcts",
)
print("GeoJSON",url)

In [ ]:
# Download Land cover rasters
url = LC2000.getDownloadURL({
    'format':"GEO_TIFF",
    'name':"GLADlandcover2000.tif",
    'region':Districts.geometry(),
    'filePerBand':True,
    'scale':30}
)
print("GEOTIFF 2000",url)

url = LC2020.getDownloadURL({
    'format':"GEO_TIFF",
    'name':"GLADlandcover2020.tif",
    'region':Districts.geometry(),
    'filePerBand':True,
    'scale':30}
)
print("GEOTIFF 2020",url)

In [ ]:
# Alternative download ee.Image for Land cover as GeoTIFF

geemap.ee_export_image(
    LC2000, filename="GLADlandcover2000.tif", scale=30, region=Districts.geometry(), file_per_band=False
)

geemap.ee_export_image(
    LC2000, filename="GLADlandcover2020.tif", scale=30, region=Districts.geometry(), file_per_band=False
)